Before you start - you need 2 things:
1. OpenAI API Key
2. App Store Connect API Key (including .p8 file)

-----
To start:
1. Click on Runtime -> Run All to start
2. Enter all information (OpenAI API Key, issuer ID etc.)
----
p.s. All information is stored only in your Google Collab.

In [ ]:
from google.colab import files

## 0. API Keys

- You need OpenAI and Appstore Connect API Keys
- Tutorial how to geth them is available here <github repo>
- Keys are not store anywhere except your Google Colab here

In [ ]:
# Initialize the OpenAI API with your API key
print("Please enter the required information:")

api_key = input("Enter OPENAI Key: ")

# APPSTORE Details
ISSUER_ID = input("Enter ISSUER_ID: ")
KEY_ID = input("Enter KEY_ID: ")
APP_ID = input("Enter APP_ID: ")
NEW_VERSION_STRING = input("Enter NEW_VERSION_STRING: ")
short_desc = input("Short Description for Whats New: ")

# To confirm the entered details (Optional)
print("\nYou entered:")
print(f"OPENAI Key: {api_key}")
print(f"ISSUER_ID: {ISSUER_ID}")
print(f"KEY_ID: {KEY_ID}")
print(f"APP_ID: {APP_ID}")
print(f"NEW_VERSION_STRING: {NEW_VERSION_STRING}")
print(f"Short Description for Whats New: {short_desc}")

## Upload your .p8 file from App Store Connect below

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():
    print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')
    SECRET_FILE_PATH = fn

## 1. Generate What's New

First, we need to generate what's new for our app

In [ ]:
import requests
import json


def generate_whats_new_description(short_description):
    """
    Generate a "What's New" description for App Store from a short description.

    Parameters:
    - short_description (str): A brief description of the app's new feature or update.

    Returns:
    - str: Generated "What's New" description.
    """
    url = 'https://api.openai.com/v1/chat/completions'

    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {api_key}'
    }

    data = {
        "model": "gpt-3.5-turbo",
        "messages": [{"role": "user", "content": f"Given the short description: '{short_description}', create a 'What's New' section for the App Store. Make it juicy. Do not use emojis"}],
        "temperature": 0.7
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        result = response.json()
        generated_description = result['choices'][0]['message']['content'].strip()
        return generated_description
    else:
        print(f"Request failed with status code {response.status_code}: {response.text}")
        return None


# Example usage
whats_new_desc_eng = generate_whats_new_description(short_desc)
print(whats_new_desc_eng)

Function below will help with translation if you have more than one languge

In [ ]:
def translate_description(locale, short_description):
    """
    Translate a short description to the given locale using ChatGPT.

    Parameters:
    - locale (str): The target language/locale (e.g., 'fr', 'es', 'de').
    - short_description (str): The description to translate.

    Returns:
    - str: Translated description, or None if translation fails.
    """
    url = 'https://api.openai.com/v1/chat/completions'

    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {api_key}'
    }

    data = {
        "model": "gpt-3.5-turbo",
        "messages": [{"role": "user", "content": f"Translate the following text to {locale}: '{short_description}'"}],
        "temperature": 0.7
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        result = response.json()
        translated_description = result['choices'][0]['message']['content'].strip()
        return translated_description
    else:
        print(f"Request failed with status code {response.status_code}: {response.text}")
        return None

# 2. Upload to App Store

## 2.1 Create New App Store Version

In [ ]:
import requests
import jwt
import time

# Constants
BASE_URL = "https://api.appstoreconnect.apple.com/v1/"

def load_secret_from_file():
    """Load the secret key content from a .p8 file."""
    with open(SECRET_FILE_PATH, 'r') as file:
        return file.read()

def create_token():
    """Generate JWT token for App Store Connect API."""
    secret_content = load_secret_from_file()
    token = jwt.encode(
        {
            'iss': ISSUER_ID,
            'exp': int(time.time()) + 1200,
            'aud': 'appstoreconnect-v1'
        },
        secret_content,
        algorithm='ES256',
        headers={'kid': KEY_ID}
    )
    return token

def create_new_app_version():
    """Create a new version for the app on App Store Connect."""
    token = create_token()
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

    # API Endpoint to create a new app version
    endpoint = BASE_URL + 'appStoreVersions'

    # Payload with details for the new version
    data = {
        "data": {
            "type": "appStoreVersions",
            "attributes": {
                "versionString": NEW_VERSION_STRING,
                "platform": "IOS"
            },
            "relationships": {
                "app": {
                    "data": {
                        "type": "apps",
                        "id": APP_ID
                    }
                }
            }
        }
    }

    response = requests.post(endpoint, headers=headers, json=data)

    if response.status_code == 201:
        version_id = response.json()['data']['id']
        print("Successfully created new app version: ", NEW_VERSION_STRING)
        print("Version ID of created app version: ", version_id)
        return version_id
    else:
        print("Failed to create new app version. Status Code:", response.status_code)
        print(response.text)
        return None

In [ ]:
def get_current_version():
    token = create_token()
    headers = {
        'Authorization': f'Bearer {token}'
    }
    url = f"https://api.appstoreconnect.apple.com/v1/apps/{APP_ID}/appStoreVersions"
    response = requests.get(url, headers=headers)
    response_data = response.json()

    print(response_data)

    # Extracting the version from the response
    if 'data' in response_data and len(response_data['data']) > 0:
        version = response_data['data'][0]['attributes']['versionString']
        return version
    else:
        return None

current_version = get_current_version()
print(current_version)

In [ ]:
version_id = create_new_app_version()

## 2.2 Get all available languages from App Store Connect

In [ ]:
def get_version_localizations(version_id):
    """Fetch the list of existing localizations for the given app version ID."""
    token = create_token()
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

    endpoint = BASE_URL + f'appStoreVersions/{version_id}/appStoreVersionLocalizations'

    response = requests.get(endpoint, headers=headers)

    print(response.json())

    if response.status_code == 200:
        localizations = response.json()['data']
        return [localization['id'] for localization in localizations], [localization['attributes']['locale'] for localization in localizations]
    else:
        print(f"Failed to fetch localizations. Status Code:", response.status_code)
        print(response.text)
        return []

In [ ]:
locale_ids, locales = get_version_localizations(version_id)

In [ ]:
print("Languages that you have: ", locales)

## 2.3 Fill All Locales you have

In [ ]:
import requests

def update_whats_new(app_localization_id, new_text):
    token = create_token()
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    url = f"https://api.appstoreconnect.apple.com/v1/appStoreVersionLocalizations/{app_localization_id}"

    data = {
        "data": {
            "type": "appStoreVersionLocalizations",
            "id": app_localization_id,
            "attributes": {
                "whatsNew": new_text
            }
        }
    }

    response = requests.patch(url, headers=headers, json=data)

    if 200 <= response.status_code < 300:
        print(f"Update successful! for id: {app_localization_id}")
        print(new_text)
        print("---------------")
    else:
        print(f"Update failed with status code {response.status_code}.")

    return response.json()

In [ ]:
locales_dict = {}

for i in range(len(locales)):
    locales_dict[locales[i]] = locale_ids[i]

In [ ]:
for locale in locales_dict:
    lang_id = locales_dict[locale]
    print(f"🌍 Starting localization for {locale} with id {lang_id}")
    WHATSNEW_DESCRIPTION = translate_description(locale, whats_new_desc_eng)
    update_whats_new(lang_id, WHATSNEW_DESCRIPTION)